# 삼성전자 종기원 시각 심화

- **Instructor**: Jongwoo Lim / Jiun Bae
- **Email**: [jlim@hanyang.ac.kr](mailto:jlim@hanyang.ac.kr) / [jiun.maydev@gmail.com](mailto:jiun.maydev@gmail.com)

## MNIST Example

In this example you will practice a simple neural network written by [TensorFlow](https://www.tensorflow.org) and [Keras](https://keras.io), using the basically used handwritten digits data set, [MNIST Dataset](http://yann.lecun.com/exdb/mnist). The goals of this example are as follows:

- Learn basically how to **write and use code**(*TensorFlow (Keras)*).
- Understand **Neural Networks** and how they work.

And this example also is written in [IPython Notebook](https://ipython.org/notebook.html), an interactive computational environment, in which you can run code directly.

### Environments

In this assignment, we assume the follows environments. 
The [Python](https://www.python.org) is a programming language that lets you work quickly and integrate systems more effectively. It is widely used in various fields, and also used in machine learning.

The [CUDA®](https://developer.nvidia.com/cuda-zone) Toolkit provides high-performance GPU-accelerated computation. In deep learning, the model takes an age to train without GPU-acceleration. ~~even with the GPU, it still takes a lot of time~~.


- [Python3](https://www.python.org/downloads/) (recommend 3.6 or above)
- [TensorFlow](https://www.tensorflow.or) is famous deep learning platform by Google (recommend 1.13 or above).
- [Keras](https://keras.io) TensorFlow 2.0 or above contain keras as high-level API.


- (Optional) [Anaconda](https://www.anaconda.com/distribution/#download-section), *popular Python Data Science Platform*
- (Optional) [PyTorch](https://pytorch.org) is an open source deep learning platform. (recommend 1.0 or above)
- (Optional) [Jupyter](https://jupyter.org/) (Notebook or Lab)
- (Optional) [CUDA](https://developer.nvidia.com/cuda-downloads) support GPU


Python packages can install by `pip install [package name]` or using **Anaconda** by `conda install [package name]`.

*If you are having trouble installing or something else, please contact TA or jiun.maydev@gmail.com.*

## Load MNIST dataset

Tesnorflow provide mnist dataset as binary archive file [link](https://chromium.googlesource.com/external/github.com/tensorflow/tensorflow/+/r0.7/tensorflow/g3doc/tutorials/mnist/download/index.md).
In this exampe, we already downloaded datafile in `./data` directory. So just laod dataset from `./data`.

In [1]:
import numpy as np
from tensorflow.python import keras

In [2]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data('mnist.npz')

## (Optional) Visualize

In [3]:
from PIL import Image
from IPython.display import display

def show(ary):
    display(Image.fromarray(ary))

In [4]:
for image, label, _ in zip(train_images, train_labels, range(5)):
    print(label)
    show(image.reshape((28, 28)))

5


0


4


1


9


## Preprocessing

The data must be preprocessed before training the network. If you inspect the first image in the training set, you will see that the pixel values fall in the range of 0 to 255. We scale these values to a range of 0 to 1 before feeding to the neural network model. For this, we divide the values by 255. It's important that the training set and the testing set are preprocessed in the same way:

In [5]:
train_images = np.expand_dims(train_images, -1)
test_images = np.expand_dims(test_images, -1)

train_images = train_images / 255.
test_images = test_images / 255.

In [6]:
from tensorflow.python.keras.utils import to_categorical
num_classes = 10
train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

## Model

In [10]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28, 1)),
    keras.layers.Dense(128, activation=keras.activations.sigmoid),
    keras.layers.Dense(10, activation=keras.activations.sigmoid)
])

## Compile

In [33]:
model.compile(optimizer='sgd',
              loss='mse',
              metrics=['accuracy'])

## Train

In [44]:
model.fit(train_images, train_labels, epochs=1)

60000/60000 [==============================] - 47s 776us/sample - loss: 0.5074 - acc: 0.8553


In [45]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Accuracy:', test_acc)

10000/10000 [==============================] - 2s 236us/sample - loss: 0.1720 - acc: 0.9485
Accuracy: 0.9485


## Predictions

In [46]:
predictions = model.predict(test_images)

In [47]:
for image, prediction, label, _ in (zip(test_images, predictions, test_labels, range(5))):
    print(f'Label: {np.argmax(label)}, Prediction: {np.argmax(prediction)}')
    show((image[:, :, 0] * 255.).astype(np.uint8))

Label: 7, Prediction: 7


Label: 2, Prediction: 2


Label: 1, Prediction: 1


Label: 0, Prediction: 0


Label: 4, Prediction: 4


## Model (with ReLU)

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=keras.activations.relu),
    keras.layers.Dense(10, activation=keras.activations.sigmoid)
])

## Model (with CNN)

In [30]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation=keras.activations.relu),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation=keras.activations.relu),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=keras.activations.relu),
    keras.layers.Dense(10, activation=keras.activations.relu),
])

## Model (with CNN, Softmax)

In [42]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation=keras.activations.relu),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation=keras.activations.relu),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=keras.activations.relu),
    keras.layers.Dense(10, activation=keras.activations.softmax),
])

In [43]:
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])